In [5]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [6]:
dataset = pd.read_csv("outwx_fine_cleaned.csv")

dataset.tail()

,ID,Lat,Lng,OutTime,Temp,Precip,WindSpd,WindGust,Press,Humid,Dew,Outage
270147,270147,-1.169530,36.668754,2020-11-05 04:00:00,18.0,0.0,10.0,12.0,1014.0,85.0,15.0,0
270148,270148,-1.329012,36.669823,2020-05-09 23:00:00,17.0,0.0,6.0,11.0,1017.0,93.0,16.0,0
270149,270149,-0.322865,37.654629,2021-04-02 04:00:00,17.0,0.0,4.0,7.0,1015.0,87.0,15.0,0
270150,270150,-1.403681,36.699726,2020-08-14 16:28:38,24.0,0.1,15.0,19.0,1010.0,52.0,13.0,1
270151,270151,-1.197639,36.844997,2020-10-20 03:00:00,18.0,0.1,11.0,13.0,1013.0,81.0,15.0,0


In [8]:
dataset = dataset.iloc[:,1:]

del dataset['OutTime']

del dataset['Lat']
del dataset['Lng']

dataset.head()




KeyError: 'OutTime'

In [4]:
import numpy as np

#lables
y_train = dataset.iloc[:,-1].values
y_train = np.array(y_train, ndmin=2)
y_train = y_train.T

print(y_train)

#features
x_train = dataset.iloc[:,:-1].values
print(x_train)

[[0]
 [0]
 [0]
 ...
 [0]
 [1]
 [0]]
[[-5.69173690e-01  3.48622741e+01  2.60000000e+01 ...  1.00800000e+03
   4.30000000e+01  1.20000000e+01]
 [ 5.94919990e-02  3.42855987e+01  2.10000000e+01 ...  1.01500000e+03
   7.10000000e+01  1.50000000e+01]
 [-1.26757657e+00  3.68424415e+01  2.10000000e+01 ...  1.01200000e+03
   7.20000000e+01  1.50000000e+01]
 ...
 [-3.22865000e-01  3.76546287e+01  1.70000000e+01 ...  1.01500000e+03
   8.70000000e+01  1.50000000e+01]
 [-1.40368056e+00  3.66997261e+01  2.40000000e+01 ...  1.01000000e+03
   5.20000000e+01  1.30000000e+01]
 [-1.19763875e+00  3.68449974e+01  1.80000000e+01 ...  1.01300000e+03
   8.10000000e+01  1.50000000e+01]]


In [5]:
#scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)

In [6]:
#splitting
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0)

In [7]:
classifier = Sequential()

# from tensorflow import keras
# from tensorflow.keras import layers

# classifier = keras.Sequential()
# classifier.add(layers.Dense(40, kernel_initializer='uniform', input_dim = 9 ))
# classifier.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
# classifier.add(Dense(30, activation='relu', kernel_initializer='random_normal'))
# classifier.add(Dense(30, activation='relu', kernel_initializer='random_normal'))
# classifier.add(Dense(50, activation='relu', kernel_initializer='random_normal'))
# classifier.add(layers.Activation('softmax'))

# opt = keras.optimizers.Adam(learning_rate=0.01)
# classifier.compile(loss='categorical_crossentropy', optimizer=opt)
    
# #First Hidden Layer
classifier.add(Dense(9, activation='relu', kernel_initializer='random_normal', input_dim=9))
classifier.add(Dense(18, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(27, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(27, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(18, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(9, activation='relu', kernel_initializer='random_normal'))

In [8]:
#Output Layer
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

# from keras.optimizers import SGD
# opt = SGD(lr=0.01)
# classifier.compile(loss = "categorical_crossentropy", optimizer = opt, metrics=['accuracy'])
# #Compiling the neural network
classifier.compile(optimizer = 'Adam', loss='binary_crossentropy', metrics =['accuracy'])



In [9]:
#Fitting the data to the training dataset
classifier.fit(x_train,y_train, batch_size=27, epochs=3, shuffle = 'TRUE')

#evaluate
eval_model=classifier.evaluate(x_test, y_test)

eval_model

Epoch 1/3
8005/8005 [==============================] - 8s 996us/step - loss: 0.1684 - accuracy: 0.9600
Epoch 2/3
8005/8005 [==============================] - 8s 1ms/step - loss: 0.1626 - accuracy: 0.9600
Epoch 3/3
1689/1689 [==============================] - 1s 754us/step - loss: 0.1642 - accuracy: 0.9594


[0.16424420475959778, 0.9593752026557922]

In [12]:
a= np.array([[-1.391075,36.767258,32,8.0,15.5,23.5,1005.0,95.0,11.0],[-0.661676,37.211036,16.0,0.2,4.0,6.0,1015.0,95.0,15.0]])
print(classifier.predict(a))

[[0.00858602]
 [0.00859252]]


In [11]:
#save model
classifier.save('final_model.h5')